# Quickstart 2 - Power Flow

## Task Description

Consider a three-zone electricity market with the following characteristics:
- Zone 1: 50 MW load, 140 MW generator with marginal cost 7.5 €/MWh, 285 MW generator with marginal cost 6 €/MWh,
- Zone 2: 60 MW load, 90 MW generator with marginal cost 14 €/MWh,
- Zone 3: 300 MW load, 85 MW generator with marginal cost 10 €/MWh.

The zones are connected by transmission lines running on 11 kV with the following characteristics:
- Line 1: connects Zone 1 and Zone 2, with a capacity of 126 MW and a reactance of 0.2 Ohm.
- Line 2: connects Zone 1 and Zone 3, with a capacity of 250 MW and a reactance of 0.2 Ohm.
- Line 3: connects Zone 2 and Zone 3, with a capacity of 130 MW and a reactance of 0.1 Ohm.

Find the least-cost dispatch of the generators to meet the loads while respecting the transmission capacities. Identify the marginal prices at each bus and the flow on the transmission lines.
Calculate the non-linear power flow based on the optimised results, and determine the losses on each line as well as the voltage angles at each bus.

## PyPSA Solution

In [ ]:
import pypsa

n = pypsa.Network()

n.add("Bus", ["zone_1", "zone_2", "zone_3"], v_nom=11)


Index(['zone_1', 'zone_2', 'zone_3'], dtype='object')

In [526]:

n.add(
    "Load",
    ["load_1", "load_2", "load_3"],
    bus=["zone_1", "zone_2", "zone_3"],
    p_set=[50, 60, 300],
)


Index(['load_1', 'load_2', 'load_3'], dtype='object')

In [527]:

n.add(
    "Generator",
    ["gen_A", "gen_B", "gen_C", "gen_D"],
    bus=["zone_1", "zone_1", "zone_2", "zone_3"],
    p_nom=[140, 285, 90, 85],
    marginal_cost=[7.5, 6, 14, 10],
)


Index(['gen_A', 'gen_B', 'gen_C', 'gen_D'], dtype='object')

In [528]:
n.add(
    "Line",
    ["line_1", "line_2", "line_3"],
    bus0=["zone_1", "zone_1", "zone_2"],
    bus1=["zone_2", "zone_3", "zone_3"],
    s_nom=[126, 250, 130],
    x=[0.02, 0.02, 0.01],
    r=0.01,
)

Index(['line_1', 'line_2', 'line_3'], dtype='object')

In [530]:
n.optimize(log_to_console=False)

Index(['zone_1', 'zone_2', 'zone_3'], dtype='object', name='Bus')
Index(['line_1', 'line_2', 'line_3'], dtype='object', name='Line')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 7 primals, 18 duals
Objective: 2.83e+03
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Kirchhoff-Voltage-Law were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


('ok', 'optimal')

In [531]:
n.model

Linopy LP model

Variables:
----------
 * Generator-p (snapshot, Generator)
 * Line-s (snapshot, Line)

Constraints:
------------
 * Generator-fix-p-lower (snapshot, Generator-fix)
 * Generator-fix-p-upper (snapshot, Generator-fix)
 * Line-fix-s-lower (snapshot, Line-fix)
 * Line-fix-s-upper (snapshot, Line-fix)
 * Bus-nodal_balance (Bus, snapshot)
 * Kirchhoff-Voltage-Law (snapshot, cycles)

Status:
-------
ok

In [532]:
n.model.constraints["Bus-nodal_balance"]

Constraint `Bus-nodal_balance` [Bus: 3, snapshot: 1]:
-----------------------------------------------------
[zone_1, now]: +1 Generator-p[now, gen_A] + 1 Generator-p[now, gen_B] - 1 Line-s[now, line_1] - 1 Line-s[now, line_2] = 50.0
[zone_2, now]: +1 Generator-p[now, gen_C] - 1 Line-s[now, line_3] + 1 Line-s[now, line_1]                             = 60.0
[zone_3, now]: +1 Generator-p[now, gen_D] + 1 Line-s[now, line_2] + 1 Line-s[now, line_3]                             = 300.0

In [533]:
n.buses_t.marginal_price

Bus,zone_1,zone_2,zone_3
snapshot,,,
now,7.5,11.25,10.0


In [534]:
n.generators_t.p

Generator,gen_A,gen_B,gen_C,gen_D
snapshot,,,,
now,50.0,285.0,-0.0,75.0


In [535]:
n.lines_t.p0

Line,line_1,line_2,line_3
snapshot,,,
now,126.0,159.0,66.0


In [536]:
n.optimize.fix_optimal_dispatch()

In [537]:
n.generators_t.p_set

Generator,gen_A,gen_B,gen_C,gen_D
snapshot,,,,
now,50.0,285.0,-0.0,75.0


In [548]:
n.generators.control

Generator
gen_A    Slack
gen_B       PQ
gen_C       PQ
gen_D       PQ
Name: control, dtype: object

In [ ]:
n.pf()

INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x762976f33e90> for snapshots Index(['now'], dtype='object', name='snapshot')


In [539]:
n.generators_t.p

Generator,gen_A,gen_B,gen_C,gen_D
snapshot,,,,
now,53.860705,285.0,-0.0,75.0


In [ ]:
(n.lines_t.p0 + n.lines_t.p1).sum().sum()

In [540]:
n.generators_t.q

Generator,gen_A,gen_B,gen_C,gen_D
snapshot,,,,
now,7.379326,0.0,0.0,0.0


In [541]:
n.lines_t.q0

,line_1,line_2,line_3
snapshot,,,
now,-1.811567,9.190894,-4.388279


In [ ]:
from numpy import pi
n.buses_t.v_ang * 180 / pi

Bus,zone_1,zone_2,zone_3
snapshot,,,
now,0.0,-1.202765,-1.532529


This example is based on Tom Brown's [Energy Systems](https://nworbmot.org/courses/es-25) course, taken from lecture [Complex Markets](https://nworbmot.org/courses/es-25/es-9-complex_markets.pdf) slides 42ff.